In [1]:
cd ..

/home/jovyan


In [2]:
%run lib/helper/load_rec_mats.py

In [3]:
%run lib/helper/load_search.py

In [4]:
from lib.recommend import get_sim
import numpy as np

In [5]:
class Recommender():
    
    def __init__(self, n_games=10000):
        self.u_vec = np.zeros(n_games)
        self.feat_bag = set()
        self.added_to_bag = set()
        self.liked = set()
        self.rated = set()
        self.item_bag = set()
        self.last_get = 0
        
    def rate(self, gs):
        self.u_vec[gs] = 1
        self.liked.add(gs)
        self.rated.add(gs)
        
    def get_bag(self, K=100):
        to_get = self.liked.difference(self.added_to_bag)
        while to_get:
            game = to_get.pop()
            top_feat_items, sim = get_sim.by_feats(game, K, feat_mat)
            self.feat_bag.update(top_feat_items)
            top_item_items, c_sim = get_sim.by_items(game, K, item_mat)
            self.item_bag.update(top_item_items)
            self.added_to_bag.add(game)
        
    def clean_bag(self):
        self.feat_bag.difference_update(self.rated)
        self.item_bag.difference_update(self.rated)
        exps = set(exp_mat[list(self.rated)].nonzero()[1])
        self.feat_bag.difference_update(exps)
        self.item_bag.difference_update(exps)
        
    def update_bag_vec(self):
        self.feat_bag_vec = np.zeros_like(self.u_vec)
        self.feat_bag_vec[list(self.feat_bag)] = 1
        self.item_bag_vec = np.zeros_like(self.u_vec)
        self.item_bag_vec[list(self.item_bag)] = 1
        
    def get_cum_vecs(self, item_weight=.7, feat_weight=.3):
        rated_sim_bag = np.multiply(item_mat[list(self.rated)], self.feat_bag_vec)
        cum_corr_vec = np.sum(rated_sim_bag, axis=0)
        self.bias_corr = np.multiply(cum_corr_vec, item_bias)
        rated_corr_bag = np.multiply(feat_mat[list(self.rated)], self.item_bag_vec)
        cum_sim_vec = np.sum(rated_corr_bag, axis=0)
        self.bias_sim = np.multiply(cum_sim_vec, item_bias)
        self.comb_vec = np.add(np.multiply(self.bias_corr, item_weight), np.multiply(self.bias_sim, feat_weight))
        
    def get(self, n):
        if len(self.rated) > self.last_get:
            self.get_bag()
            self.clean_bag()
            self.update_bag_vec()
            self.get_cum_vecs()
        comb_recs = np.argsort(self.comb_vec)[:-(n+1):-1]
        self.top_games = search_gname(comb_recs)
        _ = [print(game) for game in self.top_games]
        self.last_get = len(self.rated)

In [6]:
def search_gname(gs_list):
    '''
    Return of list of gnames from list of gs.
    '''
    return [gname[gs] for gs in gs_list]

In [27]:
New_user = Recommender()

In [14]:
New_user.rate(gs['Werewolf'])

In [24]:
New_user.rate(gs['Betrayal at House on the Hill'])

In [30]:
New_user.rate(gs['Shadowrun: Crossfire'])

In [11]:
New_user.rate(gs[''])

In [31]:
New_user.get(50)

Die Legenden von Andor: Chada & Thorn
Runebound: Sands of Al-Kalim
XenoShyft: Dreadmire
Too Many Bones
Shadowrift
The Big Book of Madness
Attack on Titan: Deck-Building Game
Shadows of Malice
Pathfinder Adventure Card Game: Wrath of the Righteous – Base Set
Warhammer Quest: The Adventure Card Game
Gloom of Kilforth: A Fantasy Quest Game
Runebound (Third Edition)
XenoShyft: Onslaught
One Deck Dungeon
Pathfinder Adventure Card Game: Mummy's Mask – Base Set
The Grizzled
Legendary Encounters: An Alien Deck Building Game
Aeon's End
Legendary Encounters: A Firefly Deck Building Game
Shadowscape
Draconis Invasion
Aeon's End: War Eternal
Apocrypha Adventure Card Game
Codex: Card-Time Strategy – Core Set
Pathfinder Adventure Card Game: Skull & Shackles – Base Set
Abduction
Citadel of Blood
Star Trek Deck Building Game: The Original Series
Legends of Andor
Darkest Night (First edition)
Legendary Encounters: A Predator Deck Building Game
Thunderstone Advance: Numenera
Space Hulk: Death Angel – Th

In [26]:
gid[gs['Last Night on Earth: Timber Peak']]

122240